# 🧪 Pipeline Testing Notebook

This notebook allows testing all components without running Streamlit.

**Structure:**
1. Setup & Imports
2. Test Models (Database)
3. Test Tools (Sanctions, Thresholds)
4. Test LLM Service
5. Test Full Pipeline (Processor)
6. Test RBAC/ABAC (Different Users)
7. Test Validation/Guardrails
8. Test Scenarios (E2E)

**Usage:** Run cells sequentially. Each section is independent after Setup.


## 1. Setup & Imports


In [1]:
# Add project root to path
import sys
from pathlib import Path

# Navigate to project root (one level up from notebooks/)
# Path().absolute() gets current directory (notebooks/), .parent gets project root
PROJECT_ROOT = Path().absolute().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")


Project root: /Users/vladislav/Documents/vlzm/kyc-analyzer/kyc-analyzer


In [2]:
# Set environment variables BEFORE imports
import os

# Required for local development
os.environ.setdefault("ENV", "LOCAL")
os.environ.setdefault("LLM_PROVIDER", "openai")  # or "openai", "azure", "anthropic"
os.environ.setdefault("OLLAMA_BASE_URL", "http://localhost:11434")
os.environ.setdefault("OLLAMA_MODEL", "llama3.2")

# Database (matches docker-compose.yml - service name is "db", not "postgres")
# Run: docker compose up -d db
# NOTE: Using = instead of setdefault() to OVERRIDE .env file values
os.environ["DATABASE_HOST"] = "localhost"
os.environ["DATABASE_PORT"] = "5432"
os.environ["DATABASE_NAME"] = "app_db"  # Matches POSTGRES_DB in docker-compose (overrides .env)
os.environ["DATABASE_USER"] = "postgres"  # Matches POSTGRES_USER in docker-compose
os.environ["DATABASE_PASSWORD"] = "localdevpassword123"

print("Environment configured:")
print(f"  ENV: {os.environ['ENV']}")
print(f"  LLM_PROVIDER: {os.environ['LLM_PROVIDER']}")
print(f"  DATABASE: {os.environ['DATABASE_NAME']} @ {os.environ['DATABASE_HOST']}:{os.environ['DATABASE_PORT']}")


Environment configured:
  ENV: LOCAL
  LLM_PROVIDER: openai
  DATABASE: app_db @ localhost:5432


In [15]:
# Core imports
from app.models import Request, RequestCreate, AnalysisResult, AnalysisOutput
from app.database import init_db, get_session
from app.services.processor import Processor
from app.services.llm_service import get_llm_service
from app.services.auth_mock import get_current_user, UserProfile, Permission, MOCK_USERS, ROLE_PERMISSIONS
from app.services.validation import run_all_validations
from app.services.tools.definitions import TOOL_DEFINITIONS, TOOL_FUNCTIONS, execute_tool

print("✅ All imports successful!")


✅ All imports successful!


## 2. Test Models (Database)


In [4]:
# Check model fields - verify your schema changes
print("Request fields:")
for name, field in Request.__fields__.items():
    print(f"  {name}: {field.annotation}")

print("\nAnalysisResult fields:")
for name, field in AnalysisResult.__fields__.items():
    print(f"  {name}: {field.annotation}")


Request fields:
  id: typing.Optional[int]
  input_text: <class 'str'>
  context: typing.Optional[str]
  group: <class 'str'>
  created_by_user_id: typing.Optional[str]
  created_at: <class 'datetime.datetime'>

AnalysisResult fields:
  id: typing.Optional[int]
  request_id: <class 'int'>
  score: <class 'int'>
  categories: list[str]
  summary: <class 'str'>
  processed_content: typing.Optional[str]
  model_version: <class 'str'>
  group: <class 'str'>
  analyzed_by_user_id: typing.Optional[str]
  llm_trace: <class 'dict'>
  human_feedback: typing.Optional[bool]
  feedback_comment: typing.Optional[str]
  feedback_by_user_id: typing.Optional[str]
  feedback_at: typing.Optional[datetime.datetime]
  validation_status: <class 'str'>
  validation_details: typing.Optional[str]
  created_at: <class 'datetime.datetime'>


/var/folders/bc/6rm76y3s2f7180l31j70ss7h0000gn/T/ipykernel_7479/571665599.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use the `model_fields` class property instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  for name, field in Request.__fields__.items():
/var/folders/bc/6rm76y3s2f7180l31j70ss7h0000gn/T/ipykernel_7479/571665599.py:7: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use the `model_fields` class property instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  for name, field in AnalysisResult.__fields__.items():


In [5]:
# Initialize database (creates tables if not exist)
init_db()
print("✅ Database initialized")

✅ Database initialized


In [6]:
# # Test creating a request manually (with rollback - won't pollute DB)
# # This is just for testing the model structure
# with get_session() as session:
#     test_request = Request(
#         input_text="Test transaction comment",
#         context="Testing from notebook",
#         group="test_group",
#     )
#     session.add(test_request)
#     session.flush()  # Get ID without committing
    
#     print(f"✅ Created request with ID: {test_request.id}")
#     print(f"   Input: {test_request.input_text}")
#     print(f"   Group: {test_request.group}")
    
#     session.rollback()
#     print("   (rolled back - test only)")


In [7]:
# Create a request that WILL be saved to DB (no rollback)
with get_session() as session:
    saved_request = Request(
        input_text="Payment for services from ABC Corp, $5,000",
        context="Test transaction saved to DB",
        group="default",
    )
    session.add(saved_request)
    # No rollback - this will be committed!
    
print(f"✅ Saved request to DB:")
print(f"   ID: {saved_request.id}")
print(f"   Input: {saved_request.input_text}")
print(f"   Group: {saved_request.group}")


✅ Saved request to DB:
   ID: 4
   Input: Payment for services from ABC Corp, $5,000
   Group: default


In [8]:
# Check what's actually in the database
from sqlmodel import select

with get_session() as session:
    # Query all requests
    stmt = select(Request)
    requests = session.exec(stmt).all()
    
    print(f"📊 Found {len(requests)} requests in database:\n")
    for req in requests:
        print(f"  ID: {req.id}")
        print(f"    Input: {req.input_text[:60]}...")
        print(f"    Group: {req.group}")
        print(f"    Created: {req.created_at}")
        print()


📊 Found 4 requests in database:

  ID: 1
    Input: Payment for services from ABC Corp, $5,000...
    Group: default
    Created: 2026-01-06 21:22:50.713721

  ID: 2
    Input: Payment for services from ABC Corp, $5,000...
    Group: default
    Created: 2026-01-06 21:23:04.259890

  ID: 3
    Input: Payment for services from ABC Corp, $5,000...
    Group: default
    Created: 2026-01-07 16:17:49.032655

  ID: 4
    Input: Payment for services from ABC Corp, $5,000...
    Group: default
    Created: 2026-01-07 16:23:21.330699



## 3. Test Tools (Function Calling)


In [9]:
# Check available tools
print(f"Registered tools: {len(TOOL_DEFINITIONS)}")

if TOOL_DEFINITIONS:
    for tool in TOOL_DEFINITIONS:
        func = tool["function"]
        print(f"\n📌 {func['name']}")
        print(f"   Description: {func['description'][:80]}...")
        print(f"   Parameters: {list(func['parameters']['properties'].keys())}")
else:
    print("ℹ️ No tools defined yet. Add tools in Phase 2.")
    print("   File: app/services/tools/definitions.py")

print(f"\nTool functions available: {list(TOOL_FUNCTIONS.keys())}")


Registered tools: 0
ℹ️ No tools defined yet. Add tools in Phase 2.
   File: app/services/tools/definitions.py

Tool functions available: ['lookup_database', 'validate_data']


In [10]:
# Test tools directly (uncomment after implementing in Phase 2)
# These tests run WITHOUT LLM - just the tool functions

# Example: Test sanctions check
# from app.services.tools.sanctions import check_sanctions_list
# result = check_sanctions_list("Ahmed Ivanov")
# print("Sanctions check result:")
# print(result)

# Example: Test threshold validation  
# from app.services.tools.thresholds import validate_amount_threshold
# result = validate_amount_threshold(9500, "USD")
# print("Threshold check result:")
# print(result)

print("ℹ️ Uncomment tool tests after implementing tools in Phase 2")


ℹ️ Uncomment tool tests after implementing tools in Phase 2


## 4. Test LLM Service



In [11]:
# Get LLM service instance
llm_service = get_llm_service()

print(f"LLM Provider: {llm_service.provider.provider_name}")
print(f"Model: {llm_service.provider.get_model_version()}")


LLM Provider: openai
Model: openai/gpt-5.2


In [12]:
# Test simple analysis (no tools)
test_input = "Payment for consulting services from John Smith, amount $5000"

print(f"Testing simple analysis...")
print(f"Input: {test_input}\n")

try:
    response = llm_service.analyze(test_input)
    print("✅ LLM Response:")
    print(f"   Score: {response.score}")
    print(f"   Categories: {response.categories}")
    print(f"   Reasoning: {response.reasoning[:200]}...")
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure LLM provider is running (ollama, openai key, etc.)")


Testing simple analysis...
Input: Payment for consulting services from John Smith, amount $5000

✅ LLM Response:
   Score: 18
   Categories: ['Financial Transaction', 'Consulting Services', 'Payment Record', 'Potential Compliance/Accounting']
   Reasoning: The input describes a straightforward business payment: a $5,000 payment for consulting services from an individual named John Smith. It contains basic transaction details (payer/payee reference, purp...


In [13]:
# Test analysis WITH tools (agent mode)
# Only works if TOOL_DEFINITIONS is not empty

if TOOL_DEFINITIONS:
    test_input = "Wire transfer from Ahmed Ivanov for $9500 USD"
    
    print(f"Testing agent mode with tools...")
    print(f"Input: {test_input}\n")
    
    try:
        response = llm_service.analyze_with_tools(test_input)
        print("✅ Agent Response:")
        print(f"   Score: {response.score}")
        print(f"   Categories: {response.categories}")
        print(f"   Tools used: {response.tools_used}")
        print(f"   Reasoning: {response.reasoning[:300]}...")
        
        if response.trace:
            print(f"\n   Trace keys: {list(response.trace.keys())}")
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("ℹ️ No tools defined yet. Agent mode test skipped.")
    print("   Define TOOL_DEFINITIONS in app/services/tools/definitions.py")


ℹ️ No tools defined yet. Agent mode test skipped.
   Define TOOL_DEFINITIONS in app/services/tools/definitions.py


## 5. Test Full Pipeline (Processor)

This is what Streamlit does behind the scenes - the complete analysis flow.


In [16]:
# Get a test user for RBAC
user = get_current_user("analyst_a")
print(f"Testing as user: {user.id} (role: {user.role})")
print(f"  Permissions: {[p.value for p in ROLE_PERMISSIONS.get(user.role, set())]}")
print(f"  Group: {user.group.value}")


Testing as user: usr_003 (role: UserRole.ANALYST)
  Permissions: ['analyze', 'view_sensitive', 'view']
  Group: group_a


In [18]:
# Run full analysis pipeline
test_data = RequestCreate(
    input_text="International wire transfer from Elena Volkova for real estate purchase, amount $150,000",
    context="High-value cross-border transaction",
    group="default",
)

print(f"Processing request...")
print(f"  Input: {test_data.input_text}")
print(f"  Context: {test_data.context}\n")

with get_session() as session:
    processor = Processor(session, user=user)
    
    try:
        request, result = processor.process_request(test_data)
        
        print("✅ Pipeline completed!")
        print(f"\n📋 Request (ID: {request.id})")
        print(f"   Input: {request.input_text[:80]}...")
        print(f"   Group: {request.group}")
        
        print(f"\n📊 Analysis Result (ID: {result.id})")
        print(f"   Score: {result.score}")
        print(f"   Categories: {result.categories}")
        print(f"   Summary: {result.summary[:200]}...")
        print(f"   Model: {result.model_version}")
        print(f"   Validation: {result.validation_status}")
        
        if result.llm_trace:
            print(f"\n🔍 LLM Trace:")
            print(f"   Keys: {list(result.llm_trace.keys())}")
            if "tools_called" in result.llm_trace:
                print(f"   Tools called: {result.llm_trace['tools_called']}")
                
    except PermissionError as e:
        print(f"❌ Permission denied: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()


Processing request...
  Input: International wire transfer from Elena Volkova for real estate purchase, amount $150,000
  Context: High-value cross-border transaction

✅ Pipeline completed!

📋 Request (ID: 5)
   Input: International wire transfer from Elena Volkova for real estate purchase, amount ...
   Group: group_a

📊 Analysis Result (ID: 1)
   Score: 62
   Categories: ['Financial Transaction', 'Cross-Border Payment', 'Real Estate Purchase', 'High-Value Transaction', 'AML/KYC Risk Indicator']
   Summary: The input describes an international wire transfer from an individual (Elena Volkova) for a real estate purchase in the amount of $150,000, explicitly noted as a high-value cross-border transaction. T...
   Model: openai/gpt-5.2
   Validation: PASS

🔍 LLM Trace:
   Keys: ['started_at', 'model', 'mode', 'input', 'completed_at', 'raw_response_preview']


In [20]:
result.llm_trace['mode']

'simple'

## 6. Test RBAC/ABAC (Different Users)

In [23]:
MOCK_USERS

{'admin_default': UserProfile(id='usr_001', username='Alice Administrator', email='alice.admin@example.com', role=<UserRole.ADMIN: 'admin'>, group=<Group.DEFAULT: 'default'>),
 'senior_default': UserProfile(id='usr_002', username='Bob Senior Analyst', email='bob.senior@example.com', role=<UserRole.SENIOR_ANALYST: 'senior_analyst'>, group=<Group.DEFAULT: 'default'>),
 'analyst_a': UserProfile(id='usr_003', username='Carol Analyst (Group A)', email='carol.analyst@example.com', role=<UserRole.ANALYST: 'analyst'>, group=<Group.GROUP_A: 'group_a'>),
 'analyst_b': UserProfile(id='usr_004', username='David Analyst (Group B)', email='david.analyst@example.com', role=<UserRole.ANALYST: 'analyst'>, group=<Group.GROUP_B: 'group_b'>),
 'viewer_a': UserProfile(id='usr_005', username='Eve Viewer (Group A)', email='eve.viewer@example.com', role=<UserRole.VIEWER: 'viewer'>, group=<Group.GROUP_A: 'group_a'>)}

In [22]:
# List all available mock users
print("Available mock users:")
for user_key, user in MOCK_USERS.items():
    print(f"\n  {user_key}:")
    print(f"    Role: {user.role}")
    print(f"  Permissions: {[p.value for p in ROLE_PERMISSIONS.get(user.role, set())]}")
    print(f"    Groups: {user.group}")


Available mock users:

  admin_default:
    Role: UserRole.ADMIN
  Permissions: ['analyze', 'view_sensitive', 'export_data', 'view_all_groups', 'manage_users', 'view']
    Groups: Group.DEFAULT

  senior_default:
    Role: UserRole.SENIOR_ANALYST
  Permissions: ['analyze', 'view_sensitive', 'export_data', 'view_all_groups', 'view']
    Groups: Group.DEFAULT

  analyst_a:
    Role: UserRole.ANALYST
  Permissions: ['analyze', 'view_sensitive', 'view']
    Groups: Group.GROUP_A

  analyst_b:
    Role: UserRole.ANALYST
  Permissions: ['analyze', 'view_sensitive', 'view']
    Groups: Group.GROUP_B

  viewer_a:
    Role: UserRole.VIEWER
  Permissions: ['view']
    Groups: Group.GROUP_A


In [29]:
from app.services.auth_mock import Permission
set[Permission]

set[app.services.auth_mock.Permission]

In [ ]:
# Test RBAC - viewer should NOT be able to analyze
from app.services.auth_mock import Permission


viewer = get_current_user("viewer_a")
print(f"Testing as VIEWER: {viewer.id}")
print(f"  Permissions: {[p.value for p in ROLE_PERMISSIONS.get(user.role, set[Permission]())]}")

with get_session() as session:
    processor = Processor(session, user=viewer)
    
    try:
        request, result = processor.process_request(RequestCreate(
            input_text="Test transaction",
        ))
        print("❌ Should have failed! Viewer shouldn't be able to analyze.")
    except PermissionError as e:
        print(f"✅ Correctly blocked: {e}")


Testing as VIEWER: usr_005
  Permissions: ['view']
✅ Correctly blocked: Access denied. User 'Eve Viewer (Group A)' with role 'viewer' does not have permission 'analyze'.


In [32]:
# Test ABAC - analysts only see their group's data
analyst_a = get_current_user("analyst_a")
analyst_b = get_current_user("analyst_b")

print(f"Analyst A groups: {analyst_a.group}")
print(f"Analyst B groups: {analyst_b.group}")

with get_session() as session:
    processor_a = Processor(session, user=analyst_a)
    results_a = processor_a.get_recent_results(limit=10)
    
    processor_b = Processor(session, user=analyst_b)
    results_b = processor_b.get_recent_results(limit=10)
    
    print(f"\nAnalyst A sees {len(results_a)} results")
    print(f"Analyst B sees {len(results_b)} results")
    
    if results_a:
        print(f"Analyst A result groups: {set(r.group for r in results_a)}")
    if results_b:
        print(f"Analyst B result groups: {set(r.group for r in results_b)}")


Analyst A groups: Group.GROUP_A
Analyst B groups: Group.GROUP_B

Analyst A sees 1 results
Analyst B sees 0 results
Analyst A result groups: {'group_a'}


## 7. Test Validation / Guardrails

In [ ]:
# Test validation functions directly
from app.services.llm_service import LLMResponse

# Create mock LLM response WITH potential PII leakage
mock_response = LLMResponse(
    score=75,
    categories=["suspicious", "high_value"],
    reasoning="This transaction shows signs of potential money laundering. The sender's SSN is 123-45-6789.",
)

original_input = "Wire transfer from John Smith"

result = run_all_validations(
    response_text=mock_response.reasoning,
    score=mock_response.score,              
    categories=mock_response.categories
)

print(f"Validation status: {result.status}")
print(f"Details: {result.details}")
print(f"Passed: {result.passed}")


Validation status: PASS
Details: None
Passed: True


## 8. Test Scenarios (E2E)

In [40]:
# Define test scenarios for the KYC/AML case
TEST_SCENARIOS = [
    {
        "name": "Clean Transaction",
        "input": "Payment for consulting services from ABC Corp, $2,500",
        "expected_risk": "LOW",
    },
    {
        "name": "Near Threshold (Structuring)",
        "input": "Cash deposit $9,500 - monthly savings",
        "expected_risk": "MEDIUM",
    },
    {
        "name": "Sanctions Match",
        "input": "Wire transfer from Ahmed Ivanov for equipment purchase, $15,000",
        "expected_risk": "CRITICAL",
    },
    {
        "name": "PEP Transaction",
        "input": "Donation from Elena Volkova for charity event, $50,000",
        "expected_risk": "HIGH",
    },
]

print(f"Defined {len(TEST_SCENARIOS)} test scenarios:")
for i, scenario in enumerate(TEST_SCENARIOS, 1):
    print(f"  {i}. {scenario['name']} - Expected: {scenario['expected_risk']}")


Defined 4 test scenarios:
  1. Clean Transaction - Expected: LOW
  2. Near Threshold (Structuring) - Expected: MEDIUM
  3. Sanctions Match - Expected: CRITICAL
  4. PEP Transaction - Expected: HIGH


In [41]:
# Run all test scenarios
def run_test_scenario(scenario: dict, user: UserProfile):
    """Run a single test scenario and return results."""
    with get_session() as session:
        processor = Processor(session, user=user)
        
        request_data = RequestCreate(
            input_text=scenario["input"],
            context=f"Test: {scenario['name']}",
        )
        
        request, result = processor.process_request(request_data)
        
        return {
            "name": scenario["name"],
            "expected": scenario["expected_risk"],
            "actual_score": result.score,
            "categories": result.categories,
            "validation": result.validation_status,
            "summary": result.summary[:100] + "...",
        }

# Run scenarios
user = get_current_user("analyst_a")

print("Running test scenarios...\n")
for scenario in TEST_SCENARIOS:
    try:
        result = run_test_scenario(scenario, user)
        
        # Determine risk level from score
        score = result["actual_score"]
        if score <= 25:
            actual_level = "LOW"
        elif score <= 50:
            actual_level = "MEDIUM"
        elif score <= 75:
            actual_level = "HIGH"
        else:
            actual_level = "CRITICAL"
        
        match = "✅" if actual_level == result["expected"] else "⚠️"
        
        print(f"{match} {result['name']}")
        print(f"   Expected: {result['expected']}, Got: {actual_level} (score: {score})")
        print(f"   Categories: {result['categories']}")
        print(f"   Validation: {result['validation']}")
        print()
        
    except Exception as e:
        print(f"❌ {scenario['name']}: {e}\n")


Running test scenarios...

✅ Clean Transaction
   Expected: LOW, Got: LOW (score: 8)
   Categories: ['Financial Transaction', 'Consulting Services', 'Business Payment', 'Low Risk / Routine Activity']
   Validation: PASS

⚠️ Near Threshold (Structuring)
   Expected: MEDIUM, Got: HIGH (score: 58)
   Categories: ['Financial transaction', 'Cash deposit', 'AML/CTF risk', 'Structuring / smurfing (near reporting threshold)', 'Transaction monitoring']
   Validation: PASS

⚠️ Sanctions Match
   Expected: CRITICAL, Got: HIGH (score: 72)
   Categories: ['Financial Transaction', 'Sanctions Screening / Potential Match', 'KYC / Counterparty Identification', 'Trade / Equipment Purchase', 'AML Risk Review']
   Validation: PASS

✅ PEP Transaction
   Expected: HIGH, Got: HIGH (score: 72)
   Categories: ['Financial Transaction', 'Charitable Donation', 'PEP (Politically Exposed Person) Screening', 'AML/CTF Risk', 'KYC/Source of Funds Verification']
   Validation: PASS



## 9. Debug Helpers

In [42]:
# Helper: View recent results from DB
from sqlmodel import select

with get_session() as session:
    stmt = select(AnalysisResult).order_by(AnalysisResult.created_at.desc()).limit(5)
    results = session.exec(stmt).all()
    
    print(f"Last {len(results)} analysis results:\n")
    for r in results:
        print(f"ID: {r.id} | Score: {r.score} | Status: {r.validation_status}")
        print(f"   Categories: {r.categories}")
        print(f"   Created: {r.created_at}")
        print()


Last 5 analysis results:

ID: 5 | Score: 72 | Status: PASS
   Categories: ['Financial Transaction', 'Charitable Donation', 'PEP (Politically Exposed Person) Screening', 'AML/CTF Risk', 'KYC/Source of Funds Verification']
   Created: 2026-01-07 19:33:13.796322

ID: 4 | Score: 72 | Status: PASS
   Categories: ['Financial Transaction', 'Sanctions Screening / Potential Match', 'KYC / Counterparty Identification', 'Trade / Equipment Purchase', 'AML Risk Review']
   Created: 2026-01-07 19:33:03.075626

ID: 3 | Score: 58 | Status: PASS
   Categories: ['Financial transaction', 'Cash deposit', 'AML/CTF risk', 'Structuring / smurfing (near reporting threshold)', 'Transaction monitoring']
   Created: 2026-01-07 19:32:50.012867

ID: 2 | Score: 8 | Status: PASS
   Categories: ['Financial Transaction', 'Consulting Services', 'Business Payment', 'Low Risk / Routine Activity']
   Created: 2026-01-07 19:32:42.380756

ID: 1 | Score: 62 | Status: PASS
   Categories: ['Financial Transaction', 'Cross-Borde

In [43]:
# Helper: View LLM trace for a specific result
import json

result_id = 1  # Change this to inspect different results

with get_session() as session:
    result = session.get(AnalysisResult, result_id)
    
    if result and result.llm_trace:
        print(f"LLM Trace for result {result_id}:")
        print(json.dumps(result.llm_trace, indent=2, default=str))
    else:
        print(f"No trace found for result {result_id}")


LLM Trace for result 1:
{
  "started_at": "2026-01-07T16:30:29.589670",
  "model": "openai/gpt-5.2",
  "mode": "simple",
  "input": {
    "input_text": "International wire transfer from Elena Volkova for real estate purchase, amount $150,000",
    "context": "High-value cross-border transaction"
  },
  "completed_at": "2026-01-07T16:30:38.013203",
  "raw_response_preview": "{\n  \"score\": 62,\n  \"categories\": [\n    \"Financial Transaction\",\n    \"Cross-Border Payment\",\n    \"Real Estate Purchase\",\n    \"High-Value Transaction\",\n    \"AML/KYC Risk Indicator\"\n  ],\n  \"summary\": \"The input describes an international wire transfer from an individual (Elena Volkova) for a real estate purchase in the amount of $150,000, explicitly noted as a high-value cross-border transaction. This combination (cross-border + real estate + relatively large amount) is a common AML/CTF risk indica"
}


In [44]:
# Helper: Clear all test data (use carefully!)
# Uncomment to run

from sqlmodel import text
with get_session() as session:
    session.exec(text("DELETE FROM analysis_results"))
    session.exec(text("DELETE FROM requests"))
    print("✅ All test data cleared")


✅ All test data cleared


---

## 📝 Quick Reference

### Before Interview:
1. `docker compose up -d db` - Start database (service name is **db**, not postgres)
2. Check LLM provider (Ollama running OR API key set)
3. Run Setup cells (1.1 - 1.3)

### During Interview - Quick Validation:
| Phase | Test Section |
|-------|-------------|
| Phase 1 (Models) | Section 2 |
| Phase 2 (Tools) | Section 3 |
| Phase 3 (Prompts) | Sections 4-5 |
| Phase 4 (Validation) | Section 7 |
| Phase 5 (Processor) | Section 5 |
| Phase 6 (UI) | Streamlit browser |

### Hotkeys:
- `Shift+Enter` - Run cell and move to next
- `Ctrl+Enter` - Run cell and stay
- `Esc + A` - Insert cell above
- `Esc + B` - Insert cell below
